If on google colab, you will need to change the hardware accelerator to GPU under Runtime > Change Runtime Type.

Then clone and install the cbmos github repository via

`!git -b dev-benchmark clone https://github.com/somathias/cbmos.git`

`%cd cbmos`

`!pip install .`


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import cbmos
import cbmos.force_functions as ff
import cbmos.solvers.euler_forward as ef
import cbmos.cell as cl

In [ ]:
# assuming GPU backend is available and accessible through CuPy
import cupy as cp

Define two models both using the forward Euler method as a solver - one using NumPy as a hpc_backend and the other using CuPy

In [2]:
dim = 2 # let's have a two-dimensional model
cbmmodel_numpy = cbmos.CBMModel(ff.Linear(), ef.solve_ivp, dim, hpc_backend=np)
cbmmodel_cupy = cbmos.CBMModel(ff.Linear(), ef.solve_ivp, dim, hpc_backend=cp)

Let's generate a large set of random numbers...

In [19]:
seed = 845
np.random.seed(seed) # set seed for reproducibility if necessary
n_x = 40
n_y = 40
scl = 0.8
xcrds = [(2 * i + (j % 2)) * 0.5 * scl for j in range(n_y) for i in range(n_x)]
ycrds = [np.sqrt(3) * j * 0.5 * scl for j in range(n_y) for i in range(n_x)]

...which we can use as initial coordinates of our (non-proliferating) cell population.

In [20]:
cell_list = []
for i, (x, y) in enumerate(zip(xcrds, ycrds)):
    cell_list.append(cl.Cell(i, [x, y], proliferating=False))

Now time the models on the mechanical relaxation of our random cell population.

In [21]:
t_data = np.linspace(0, 1, 101)

In [22]:
%time history = cbmmodel_numpy.simulate(cell_list, t_data, {}, {"dt": 0.01}, seed=seed, box=True)

CPU times: user 2min 47s, sys: 455 ms, total: 2min 48s
Wall time: 2min 50s


In [23]:
%time history = cbmmodel_numpy.simulate(cell_list, t_data, {}, {"dt": 0.01}, seed=seed)

CPU times: user 21.5 s, sys: 4.23 s, total: 25.8 s
Wall time: 26.1 s


In [ ]:
%time history = cbmmodel_cupy.simulate(cell_list, t_data, {}, {"dt": 0.01}, seed=seed)